In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
version2inferfile_tax = {
    'emb_v0.0': "../../models/product_title_embedding/version_0/epoch=4-step=370604--wish-newtax-v1.2.1.json", 
    'emb_v0.4': "../../models/product_title_embedding/version_4/epoch=4-step=398293--wish-newtax-v1.2.1.json"
}

In [3]:
version2inferfile = {
    'clm_v0.4_aliprompt': "../../models/product_title_taxonomy_classification/version_4/epoch=0-step=996156--wish-offshore-validated-test-aliprompt.json",
    'clm_v0.4_wishprompt': "../../models/product_title_taxonomy_classification/version_4/epoch=0-step=996156--wish-offshore-validated-test-wishprompt.json",
    'clm_v0.5_aliprompt': "../../models/product_title_taxonomy_classification/version_5/epoch=1-step=1006577--wish-offshore-validated-test-aliprompt.json",
    'clm_v0.5_wishprompt': "../../models/product_title_taxonomy_classification/version_5/epoch=1-step=1006577--wish-offshore-validated-test-wishprompt.json",
    'emb_v0.0': "../../models/product_title_embedding/version_0/epoch=4-step=370604--wish-offshore-validated-test.json",
    'emb_v0.4': "../../models/product_title_embedding/version_4/epoch=4-step=398293--wish-offshore-validated-test.json",
    'seqclf_v0.0': "../../models/product_title_taxonomy_sequence_classification/version_0/seqclf-epoch=4-step=59300--wish-offshore-validated-test.json"
}

In [4]:
version2distance_func = {
    'emb_v0.0': 'cosine',
    'emb_v0.4': 'order'
}

In [5]:
version2decode_method = {
    'seqclf_v0.0': 'leaf'
}

In [6]:
from eval_from_file import perf_eval_util, get_df_outfile, read_json_helper
from eval_from_file_emb import perf_eval_util_emb
from eval_from_file_seqclf import perf_eval_utils_seqclf

In [7]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [8]:
import torch 
torch.cuda.device_count()

1

In [9]:
dataset = "offshore-validated"

In [10]:
label_map_file = "../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt"

In [11]:
filter_data_func = lambda df_data: df_data[df_data.category.apply(lambda x: x[0] not in ['apparel accessories', 'automobiles & motorcycles'])]

# eval

In [12]:
filtered_perms = []
ind = -1
for version in tqdm(version2inferfile):
    ind += 1
    if version.startswith('clm'):
        infer_path = version2inferfile[version]
        df_data, _ = get_df_outfile(dataset)
        df_pred = read_json_helper(infer_path)
        df_pred = df_pred[df_pred.rank_indices == 0].sort_values('batch_indices')
        assert len(df_data) == len(df_pred)
        df_data[f'{version}_predicted_category'] = df_pred['prediction_decoded'].tolist()
        df_data[f'{version}_predicted_category'] = df_data[f'{version}_predicted_category'].apply(lambda x: x.split(' > '))
        df_data = filter_data_func(df_data)
        perm = pd.concat([perf_eval_util(df_data, level=i, col=f'{version}_predicted_category') for i in [1, 2, 0, -1, -2]])
        perm['version'] = version
        filtered_perms += perm[(perm.level == 1) & (perm['id'] == 'weighted avg')].to_dict('records')
        if ind == 0:
            perm_lance = pd.concat([perf_eval_util(df_data, level=i, col='lance_predicted_category') for i in [1, 2, 0, -1, -2]])
            perm_lance['version'] = 'lance'
            filtered_perms += perm_lance[(perm_lance.level == 1) & (perm_lance['id'] == 'weighted avg')].to_dict('records')
    elif version.startswith('emb'):
        infer_path_title = version2inferfile[version]
        infer_path_tax = version2inferfile_tax[version]
        df_data, _ = get_df_outfile(dataset) 
        df_pred = read_json_helper(infer_path_title) # title
        df_pred = df_pred.sort_values('batch_indices')
        assert len(df_data) == len(df_pred), f"df_data {len(df_data)} df_pred {len(df_pred)}"
        df_pred_tax = read_json_helper(infer_path_tax) # tax
        distance_func = version2distance_func[version]
        df_data = perf_eval_util_emb(df_pred, df_pred_tax, df_data, version, distance_func)
        df_data = filter_data_func(df_data)
        perm = pd.concat([perf_eval_util(df_data, level=i, col=f'{version}_predicted_category') for i in [1, 2, 0, -1, -2]])
        perm['version'] = version
        filtered_perms += perm[(perm.level == 1) & (perm['id'] == 'weighted avg')].to_dict('records')
    elif version.startswith('seqclf'):
        infer_path = version2inferfile[version]
        df_pred = read_json_helper(infer_path)
        df_data, _ = get_df_outfile(dataset) 
        df_pred = df_pred.sort_values('batch_indices')
        assert len(df_data) == len(df_pred), f"df_data {len(df_data)} df_pred {len(df_pred)}"
        decode_method = version2decode_method[version]
        df_data = perf_eval_utils_seqclf(df_pred, df_data, label_map_file, version, decode_method)
        df_data = filter_data_func(df_data)
        perm = pd.concat([perf_eval_util(df_data, level=i, col=f'{version}_predicted_category') for i in [1, 2, 0, -1, -2]])
        perm['version'] = version
        filtered_perms += perm[(perm.level == 1) & (perm['id'] == 'weighted avg')].to_dict('records')
    else:
        raise NotImplemented()
df_filtered_perms = pd.DataFrame(filtered_perms)

 57%|█████▋    | 4/7 [00:14<00:09,  3.24s/it]/workspaces/multitask-llm-rnd/modelling/notebooks/eval/eval_from_file.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tmp_model'] = df[model].apply(lambda x: x[:level] if len(x[:level]) > 0 else x[:1])
/workspaces/multitask-llm-rnd/modelling/notebooks/eval/eval_from_file.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tmp_label'] = df[labeler].apply(lambda x: x[:level] if len(x[:level]) > 0 else x[:1])
/workspaces/multitask-llm-rnd/modelling/not

In [15]:
df_filtered_perms.sort_values('version').to_csv('tmp_one_off_eval.csv', index=False)

In [17]:
df_filtered_perms.sort_values('version')

,id,precision,recall,f1-score,support,level,version
0,weighted avg,0.817401,0.787657,0.791251,956.0,1,clm_v0.4_aliprompt
2,weighted avg,0.814522,0.783473,0.787064,956.0,1,clm_v0.4_wishprompt
3,weighted avg,0.803159,0.752092,0.749196,956.0,1,clm_v0.5_aliprompt
4,weighted avg,0.805846,0.750000,0.745798,956.0,1,clm_v0.5_wishprompt
5,weighted avg,0.794184,0.769874,0.773274,956.0,1,emb_v0.0
6,weighted avg,0.803957,0.778243,0.781252,956.0,1,emb_v0.4
1,weighted avg,0.830450,0.811715,0.807202,956.0,1,lance
7,weighted avg,0.827917,0.807531,0.807520,956.0,1,seqclf_v0.0
